# Projeto Magazine Luiza

## Importando bibliotecas

In [1]:
from selenium import webdriver #pode ser utilizado em navegadores
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from sqlalchemy import create_engine 


## Site Magazine Luiza

In [2]:
navegador = webdriver.Chrome() #comando para abrir o navegador
navegador.get('https://www.magazineluiza.com.br/selecao/ofertasdodia/?filters=category---MO')

## DF Móveis

### Extração e criação do DF Móveis

In [3]:
# Listas para armazenar os dados dos móveis
lista_moveis = []
tipo_moveis = []
lista_precos_pix_moveis = []
lista_precos_bruto_moveis = []
lista_avaliacao_moveis = []

# Preenchendo as listas com os dados dos móveis
for i in range(1, 31):
    dado_movel = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/h2').text
    lista_moveis.append(dado_movel)
    tipo_moveis.append("Móvel")
    
    try:
        dado_preco_pix_moveis = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/div/p').text
    except:
        dado_preco_pix_moveis = '0'
    lista_precos_pix_moveis.append(dado_preco_pix_moveis)

    try:
        dado_avaliacao_moveis = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[2]/div/span').text
    except:
        dado_avaliacao_moveis = '0'
    lista_avaliacao_moveis.append(dado_avaliacao_moveis)
    
    try:
        dado_preco_bruto_moveis = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/p').text
    except:
        try:
            dado_preco_bruto_moveis = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[4]/div/div/p').text
        except:
            dado_preco_bruto_moveis = dado_preco_pix_moveis 
    
    lista_precos_bruto_moveis.append(dado_preco_bruto_moveis)

# Criando o DataFrame para Móveis
df_moveis = pd.DataFrame({
    "Produto": lista_moveis,
    "Tipo": tipo_moveis,
    "Preço (Pix)": lista_precos_pix_moveis,
    "Preço Bruto": lista_precos_bruto_moveis,
    "Avaliação": lista_avaliacao_moveis
})

print(df_moveis)

                                              Produto   Tipo     Preço (Pix)  \
0   Cama Box Casal Cinza + Colchão Molas Ensacadas...  Móvel    ou R$ 730,31   
1   Sofá 4 Lugares Net Jobim Assento Retrátil e Re...  Móvel  ou R$ 1.393,41   
2   Rack com Painel TV 65" Retrô Flórida Up Multim...  Móvel    ou R$ 358,94   
3   Armário de Cozinha Completa 260cm Reims Madesa 09  Móvel  ou R$ 1.234,99   
4   Mesa de Jantar 1,70 Apogeu com vidro Serigrafa...  Móvel  ou R$ 1.116,00   
5            Balcão de Cozinha Colormaq com Tampo Aço  Móvel    ou R$ 468,73   
6   Berço Com Colchão Ditália Multifuncional 3x1 B...  Móvel    ou R$ 458,55   
7   Armário de Cozinha Diana 100% MDF Tuboarte 8 P...  Móvel  ou R$ 1.342,34   
8   Guarda-Roupa Casal Honey 4 Portas 6 Gavetas co...  Móvel               0   
9   Kit Armário De Cozinha Suspensa Juliete Com 7 ...  Móvel    ou R$ 414,11   
10          Cama Solteiro Bicama Ditália BB-28 Branco  Móvel    ou R$ 315,09   
11  Sala de Jantar Mesa Olímpia 120CM MD

### Tratamento DF Móveis

In [4]:
df_moveis['Preço (Pix)'] = df_moveis['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
df_moveis['Avaliação'] = df_moveis['Avaliação'].str.slice(0, 3).astype(float)
df_moveis['Preço Bruto'] = df_moveis['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)
df_moveis['Desconto'] = (df_moveis['Preço Bruto'] - df_moveis['Preço (Pix)']) / df_moveis['Preço Bruto']
df_moveis['Desconto'] = df_moveis['Desconto'].round(2)
df_moveis = df_moveis[df_moveis['Desconto'] >= 0]

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\1522575510.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_moveis['Preço (Pix)'] = df_moveis['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\1522575510.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_moveis['Preço Bruto'] = df_moveis['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)


### DF Móveis

In [5]:
df_moveis

,Produto,Tipo,Preço (Pix),Preço Bruto,Avaliação,Desconto
1,Sofá 4 Lugares Net Jobim Assento Retrátil e Re...,Móvel,393.41,887.99,4.1,0.56
2,"Rack com Painel TV 65"" Retrô Flórida Up Multim...",Móvel,358.94,398.82,4.6,0.10
3,Armário de Cozinha Completa 260cm Reims Madesa 09,Móvel,234.99,299.99,4.6,0.22
4,"Mesa de Jantar 1,70 Apogeu com vidro Serigrafa...",Móvel,116.00,799.90,4.6,0.85
5,Balcão de Cozinha Colormaq com Tampo Aço,Móvel,468.73,899.99,4.5,0.48
6,Berço Com Colchão Ditália Multifuncional 3x1 B...,Móvel,458.55,509.50,4.4,0.10
7,Armário de Cozinha Diana 100% MDF Tuboarte 8 P...,Móvel,342.34,875.69,4.3,0.61
9,Kit Armário De Cozinha Suspensa Juliete Com 7 ...,Móvel,414.11,500.00,4.2,0.17
10,Cama Solteiro Bicama Ditália BB-28 Branco,Móvel,315.09,509.50,4.3,0.38
12,Rack com Painel para TV Até 72 Polegadas Vivar...,Móvel,30.90,975.23,4.3,0.97


## DF Utilidades Domésticas

In [6]:
navegador.get('https://www.magazineluiza.com.br/selecao/ofertasdodia/?filters=category---UD')

### Extração e criação do DF Utilidades Domésticas

In [7]:
# Listas para armazenar os dados das utilidades domésticas
lista_utilidades = []
tipo_utilidades = []
lista_precos_pix_utilidades = []
lista_precos_bruto_utilidades = []
lista_avaliacao_utilidades = []

# Preenchendo as listas com os dados das utilidades domésticas
for i in range(1, 31): 
    dado_utilidade = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/h2').text
    lista_utilidades.append(dado_utilidade)
    tipo_utilidades.append("Utilidade Doméstica")
    
    try:
        dado_preco_pix_utilidades = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/div/p').text
    except:
        dado_preco_pix_utilidades = '0'
    lista_precos_pix_utilidades.append(dado_preco_pix_utilidades)

    try:
        dado_avaliacao_utilidades = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[2]/div/span').text
    except:
        dado_avaliacao_utilidades = '0'
    lista_avaliacao_utilidades.append(dado_avaliacao_utilidades)
    
    try:
        dado_preco_bruto_utilidades = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/p').text
    except:
        try:
            dado_preco_bruto_utilidades = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[4]/div/div/p').text
        except:
            dado_preco_bruto_utilidades = dado_preco_pix_utilidades 
    
    lista_precos_bruto_utilidades.append(dado_preco_bruto_utilidades)

# Criando o DataFrame para Utilidades Domésticas
df_utilidades = pd.DataFrame({
    "Produto": lista_utilidades,
    "Tipo": tipo_utilidades,
    "Preço (Pix)": lista_precos_pix_utilidades,
    "Preço Bruto": lista_precos_bruto_utilidades,
    "Avaliação": lista_avaliacao_utilidades
})

print(df_utilidades)

                                              Produto                 Tipo  \
0   Porta temperos e Condimentos Giratório Com 9 P...  Utilidade Doméstica   
1          Jogo C/ 6 Taças Vidro Diamond Transparente  Utilidade Doméstica   
2   Escorredor De Louca Cozinha 13 Pratos Copo Dup...  Utilidade Doméstica   
3   Kit Jogo 12 Peças Utensílios De Silicone Para ...  Utilidade Doméstica   
4          Cuscuzeiro Individual E Cozi Vapore Polido  Utilidade Doméstica   
5    Jogo De Panelas Antiaderente Preto Arnix 6 Peças  Utilidade Doméstica   
6   Cuscuzeiro De Alumínio Panela Cuscuzeiro Cuscu...  Utilidade Doméstica   
7   Bandeja com 2 Gavetas Porta Cápsulas Café Dolc...  Utilidade Doméstica   
8       Cuscuzeiro Individual Nordestino E Cozi Vapor  Utilidade Doméstica   
9   Conjunto De Frigideiras 3 Peças Antiaderente T...  Utilidade Doméstica   
10  Panela De Pressão 4,5 Litros Vancouver Tramontina  Utilidade Doméstica   
11  Forma De Bolo Pudim Com Tubo Em Alumínio 22 X ...  Utilidade

### Tratamento Utilidades Domésticas

In [8]:
# Tratamento de Dados
df_utilidades['Preço (Pix)'] = df_utilidades['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
df_utilidades['Avaliação'] = df_utilidades['Avaliação'].str.slice(0, 3).astype(float)
df_utilidades['Preço Bruto'] = df_utilidades['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)
df_utilidades['Desconto'] = (df_utilidades['Preço Bruto'] - df_utilidades['Preço (Pix)']) / df_utilidades['Preço Bruto']
df_utilidades['Desconto'] = df_utilidades['Desconto'].round(2)
df_utilidades = df_utilidades[df_utilidades['Desconto'] >= 0]

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\1453406692.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_utilidades['Preço (Pix)'] = df_utilidades['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\1453406692.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_utilidades['Preço Bruto'] = df_utilidades['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)


In [9]:
df_utilidades

,Produto,Tipo,Preço (Pix),Preço Bruto,Avaliação,Desconto
0,Porta temperos e Condimentos Giratório Com 9 P...,Utilidade Doméstica,47.59,78.99,4.7,0.40
1,Jogo C/ 6 Taças Vidro Diamond Transparente,Utilidade Doméstica,42.39,42.39,4.8,0.00
2,Escorredor De Louca Cozinha 13 Pratos Copo Dup...,Utilidade Doméstica,37.19,37.19,4.5,0.00
3,Kit Jogo 12 Peças Utensílios De Silicone Para ...,Utilidade Doméstica,56.00,56.00,4.7,0.00
4,Cuscuzeiro Individual E Cozi Vapore Polido,Utilidade Doméstica,24.57,24.57,4.5,0.00
5,Jogo De Panelas Antiaderente Preto Arnix 6 Peças,Utilidade Doméstica,88.05,119.90,4.5,0.27
6,Cuscuzeiro De Alumínio Panela Cuscuzeiro Cuscu...,Utilidade Doméstica,25.00,33.99,4.3,0.26
7,Bandeja com 2 Gavetas Porta Cápsulas Café Dolc...,Utilidade Doméstica,59.47,78.99,4.7,0.25
8,Cuscuzeiro Individual Nordestino E Cozi Vapor,Utilidade Doméstica,24.57,24.57,4.6,0.00
9,Conjunto De Frigideiras 3 Peças Antiaderente T...,Utilidade Doméstica,40.63,40.63,4.5,0.00


## DF Esporte e Lazer

In [10]:
navegador.get('https://www.magazineluiza.com.br/selecao/ofertasdodia/?filters=category---ES')

### Extração e criação do DF Esportes e Lazer

In [11]:
lista_esportes = []
tipo_esportes = []
lista_precos_pix_esportes = []
lista_precos_bruto_esportes = []
lista_avaliacao_esportes = []

# Preenchendo as listas com os dados de Esportes
for i in range(1, 31):  # Ajuste o número de produtos conforme necessário
    dado_esporte = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/h2').text
    lista_esportes.append(dado_esporte)
    tipo_esportes.append("Esporte")
    
    try:
        dado_preco_pix_esportes = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/div/p').text
    except:
        dado_preco_pix_esportes = '0'
    lista_precos_pix_esportes.append(dado_preco_pix_esportes)

    try:
        dado_avaliacao_esportes = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[2]/div/span').text
    except:
        dado_avaliacao_esportes = '0'
    lista_avaliacao_esportes.append(dado_avaliacao_esportes)
    
    try:
        dado_preco_bruto_esportes = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/p').text
    except:
        try:
            dado_preco_bruto_esportes = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[4]/div/div/p').text
        except:
            dado_preco_bruto_esportes = dado_preco_pix_esportes 
    
    lista_precos_bruto_esportes.append(dado_preco_bruto_esportes)

# Criando o DataFrame para Esportes
df_esportes = pd.DataFrame({
    "Produto": lista_esportes,
    "Tipo": tipo_esportes,
    "Preço (Pix)": lista_precos_pix_esportes,
    "Preço Bruto": lista_precos_bruto_esportes,
    "Avaliação": lista_avaliacao_esportes
})

print(df_esportes)

                                              Produto     Tipo  \
0                      Tênis Adidas Breaknet Feminino  Esporte   
1         Caixa Térmica Cooler Suv 20 Litros Termolar  Esporte   
2                     Tênis Adidas Breaknet Masculino  Esporte   
3   Kit 2 malas de viagem de bordo PP e P Detroit ...  Esporte   
4   Bicicleta infantil Feminina Aro 16 Love Tiffan...  Esporte   
5                  Tênis Adidas Ultra Energy Feminino  Esporte   
6                            Tenis Adidas Switch Move  Esporte   
7   Fogão Cooktop De Mesa Portátil A Gás 4 Bocas B...  Esporte   
8                          Tênis Mizuno Goya Feminino  Esporte   
9                    Tênis Olympikus Estilo 2 Unissex  Esporte   
10  Bicicleta Ergométrica Magnética Horizontal Gal...  Esporte   
11                    Tênis Mizuno Raijuu 2 Masculino  Esporte   
12                           Tenis Adidas Switch Move  Esporte   
13                  Tênis Olympikus Estilo 2 Feminino  Esporte   
14  Bicicl

### Tratamento Esporte e Lazer

In [12]:
# Tratamento de Dados
df_esportes['Preço (Pix)'] = df_esportes['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
df_esportes['Avaliação'] = df_esportes['Avaliação'].str.slice(0, 3).astype(float)
df_esportes['Preço Bruto'] = df_esportes['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)
df_esportes['Desconto'] = (df_esportes['Preço Bruto'] - df_esportes['Preço (Pix)']) / df_esportes['Preço Bruto']
df_esportes['Desconto'] = df_esportes['Desconto'].round(2)
df_esportes = df_esportes[df_esportes['Desconto'] >= 0]

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\3133416862.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_esportes['Preço (Pix)'] = df_esportes['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\3133416862.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_esportes['Preço Bruto'] = df_esportes['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)


In [13]:
df_esportes

,Produto,Tipo,Preço (Pix),Preço Bruto,Avaliação,Desconto
0,Tênis Adidas Breaknet Feminino,Esporte,199.99,199.99,4.6,0.00
1,Caixa Térmica Cooler Suv 20 Litros Termolar,Esporte,120.12,133.47,4.8,0.10
2,Tênis Adidas Breaknet Masculino,Esporte,199.99,349.99,4.7,0.43
3,Kit 2 malas de viagem de bordo PP e P Detroit ...,Esporte,249.90,443.61,4.8,0.44
4,Bicicleta infantil Feminina Aro 16 Love Tiffan...,Esporte,572.84,749.98,4.9,0.24
5,Tênis Adidas Ultra Energy Feminino,Esporte,199.99,199.99,4.7,0.00
6,Tenis Adidas Switch Move,Esporte,189.90,219.99,4.8,0.14
7,Fogão Cooktop De Mesa Portátil A Gás 4 Bocas B...,Esporte,199.90,299.90,4.3,0.33
8,Tênis Mizuno Goya Feminino,Esporte,179.99,329.99,5.0,0.45
9,Tênis Olympikus Estilo 2 Unissex,Esporte,134.99,259.99,4.9,0.48


## DF Casa e Construção

In [14]:
navegador.get('https://www.magazineluiza.com.br/selecao/ofertasdodia/?filters=category---CJ')

### Extração e criação do DF Casa e Construção

In [15]:
# Listas para armazenar os dados de Casa e Construção
lista_casa_construção = []
tipo_casa_construção = []
lista_precos_pix_casa_construção = []
lista_precos_bruto_casa_construção = []
lista_avaliacao_casa_construção = []

# Preenchendo as listas com os dados de Casa e Construção
for i in range(1, 31):
    dado_casa_construção = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/h2').text
    lista_casa_construção.append(dado_casa_construção)
    tipo_casa_construção.append("Casa e Construção")
    
    try:
        dado_preco_pix_casa_construção = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/div/p').text
    except:
        dado_preco_pix_casa_construção = '0'
    lista_precos_pix_casa_construção.append(dado_preco_pix_casa_construção)

    try:
        dado_avaliacao_casa_construção = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[2]/div/span').text
    except:
        dado_avaliacao_casa_construção = '0'
    lista_avaliacao_casa_construção.append(dado_avaliacao_casa_construção)
    
    try:
        dado_preco_bruto_casa_construção = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[3]/div/p').text
    except:
        try:
            dado_preco_bruto_casa_construção = navegador.find_element(By.XPATH, f'//*[@id="__next"]/div/main/section[3]/div[2]/div/ul/li[{i}]/a/div[3]/div[4]/div/div/p').text
        except:
            dado_preco_bruto_casa_construção = dado_preco_pix_casa_construção 
    
    lista_precos_bruto_casa_construção.append(dado_preco_bruto_casa_construção)

# Criando o DataFrame para Casa e Construção
df_casa_construção = pd.DataFrame({
    "Produto": lista_casa_construção,
    "Tipo": tipo_casa_construção,
    "Preço (Pix)": lista_precos_pix_casa_construção,
    "Preço Bruto": lista_precos_bruto_casa_construção,
    "Avaliação": lista_avaliacao_casa_construção
})

print(df_casa_construção)

                                              Produto               Tipo  \
0   Câmera Externa Ip Prova D'água Infravermelho E...  Casa e Construção   
1   Jogo de Cama Queen 100% Poliéster Marcelino Tê...  Casa e Construção   
2   Tampa Pra Vaso Sanitário Almofadado Deca Icasa...  Casa e Construção   
3            Cuba Pia para Banheiro Retangular Torino  Casa e Construção   
4   Cuba Pia Gourmet Luxo Prizi Aço Inox 201 42x60...  Casa e Construção   
5   Escada De Alumínio 3 Degraus Tesoura Capacidad...  Casa e Construção   
6   Ralo Inteligente Click Para Banheiro 15x15 Cm ...  Casa e Construção   
7    Luminária De Mesa Pixar Articulável Com Base E27  Casa e Construção   
8       Hidrosan Penta 10 Kg Cloro Granulado Hidroall  Casa e Construção   
9   HTH PURIFICAR CLORO CONCENTRADO HIPOCLORITO PA...  Casa e Construção   
10  Torneira Banheiro Inox Giratória Mesa Curta NO...  Casa e Construção   
11  Válvula Pia Cozinha 4 1/2 Inox 304 Com Cesto R...  Casa e Construção   
12  Cuba par

### Extração Tratamento Casa e Construção

In [16]:
# Tratamento de Dados
df_casa_construção['Preço (Pix)'] = df_casa_construção['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
df_casa_construção['Avaliação'] = df_casa_construção['Avaliação'].str.slice(0, 3).astype(float)
df_casa_construção['Preço Bruto'] = df_casa_construção['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)
df_casa_construção['Desconto'] = (df_casa_construção['Preço Bruto'] - df_casa_construção['Preço (Pix)']) / df_casa_construção['Preço Bruto']
df_casa_construção['Desconto'] = df_casa_construção['Desconto'].round(2)
df_casa_construção = df_casa_construção[df_casa_construção['Desconto'] >= 0]

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\3619789291.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_casa_construção['Preço (Pix)'] = df_casa_construção['Preço (Pix)'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)  # Converte para tipo float
C:\Users\Lucca Cardozo\AppData\Local\Temp\ipykernel_11160\3619789291.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_casa_construção['Preço Bruto'] = df_casa_construção['Preço Bruto'].str.extract('(\d+,\d+)')[0].str.replace(',', '.').astype(float)


In [17]:
df_casa_construção

,Produto,Tipo,Preço (Pix),Preço Bruto,Avaliação,Desconto
0,Câmera Externa Ip Prova D'água Infravermelho E...,Casa e Construção,123.01,235.00,4.4,0.48
1,Jogo de Cama Queen 100% Poliéster Marcelino Tê...,Casa e Construção,39.10,99.90,4.4,0.61
2,Tampa Pra Vaso Sanitário Almofadado Deca Icasa...,Casa e Construção,69.99,69.99,4.3,0.00
3,Cuba Pia para Banheiro Retangular Torino,Casa e Construção,119.22,143.99,4.6,0.17
4,Cuba Pia Gourmet Luxo Prizi Aço Inox 201 42x60...,Casa e Construção,89.02,109.90,5.0,0.19
5,Escada De Alumínio 3 Degraus Tesoura Capacidad...,Casa e Construção,109.91,124.90,4.8,0.12
7,Luminária De Mesa Pixar Articulável Com Base E27,Casa e Construção,55.20,78.99,4.5,0.30
8,Hidrosan Penta 10 Kg Cloro Granulado Hidroall,Casa e Construção,240.35,279.00,4.8,0.14
9,HTH PURIFICAR CLORO CONCENTRADO HIPOCLORITO PA...,Casa e Construção,350.55,369.01,4.7,0.05
10,Torneira Banheiro Inox Giratória Mesa Curta NO...,Casa e Construção,50.07,69.90,4.7,0.28


## DF Magalu

In [18]:
df_magalu = pd.concat([df_utilidades, df_moveis, df_casa_construção, df_esportes])

In [19]:
df_magalu


,Produto,Tipo,Preço (Pix),Preço Bruto,Avaliação,Desconto
0,Porta temperos e Condimentos Giratório Com 9 P...,Utilidade Doméstica,47.59,78.99,4.7,0.40
1,Jogo C/ 6 Taças Vidro Diamond Transparente,Utilidade Doméstica,42.39,42.39,4.8,0.00
2,Escorredor De Louca Cozinha 13 Pratos Copo Dup...,Utilidade Doméstica,37.19,37.19,4.5,0.00
3,Kit Jogo 12 Peças Utensílios De Silicone Para ...,Utilidade Doméstica,56.00,56.00,4.7,0.00
4,Cuscuzeiro Individual E Cozi Vapore Polido,Utilidade Doméstica,24.57,24.57,4.5,0.00
...,...,...,...,...,...,...
25,Tênis Adidas Ultra Energy Feminino,Esporte,219.99,219.99,4.7,0.00
26,Tênis Masculino Adidas Ultra Energy,Esporte,199.99,199.99,4.5,0.00
27,Kit Bico de Bomba 5 Peças Adaptador Agulhas Bo...,Esporte,4.90,7.90,4.5,0.38
28,Tênis Mizuno Goya Masculino,Esporte,179.99,329.99,4.9,0.45


## Exportando em CSV

In [20]:
df_magalu.to_csv('../1_bases_tratadas/base_magalu.csv', sep=';', encoding='utf-8-sig')

In [21]:
engine = create_engine('sqlite:///banco.db', echo=True)
df_magalu.to_sql('Magazine', con=engine, if_exists='replace', index=False)
df_lido = pd.read_sql('SELECT * FROM Magazine', con=engine)

2024-11-25 21:49:22,302 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-25 21:49:22,311 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Magazine")
2024-11-25 21:49:22,313 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-25 21:49:22,317 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Magazine")
2024-11-25 21:49:22,319 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-25 21:49:22,326 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Magazine" (
	"Produto" TEXT, 
	"Tipo" TEXT, 
	"Preço (Pix)" FLOAT, 
	"Preço Bruto" FLOAT, 
	"Avaliação" FLOAT, 
	"Desconto" FLOAT
)


2024-11-25 21:49:22,330 INFO sqlalchemy.engine.Engine [no key 0.00409s] ()
2024-11-25 21:49:24,696 INFO sqlalchemy.engine.Engine INSERT INTO "Magazine" ("Produto", "Tipo", "Preço (Pix)", "Preço Bruto", "Avaliação", "Desconto") VALUES (?, ?, ?, ?, ?, ?)
2024-11-25 21:49:24,697 INFO sqlalchemy.engine.Engine [generated in 0.00206s] [('Porta temperos e Condimentos Giratório Com 9 Potes de vidro Preto', 'Utilidade D

In [22]:
df_lido

,Produto,Tipo,Preço (Pix),Preço Bruto,Avaliação,Desconto
0,Porta temperos e Condimentos Giratório Com 9 P...,Utilidade Doméstica,47.59,78.99,4.7,0.40
1,Jogo C/ 6 Taças Vidro Diamond Transparente,Utilidade Doméstica,42.39,42.39,4.8,0.00
2,Escorredor De Louca Cozinha 13 Pratos Copo Dup...,Utilidade Doméstica,37.19,37.19,4.5,0.00
3,Kit Jogo 12 Peças Utensílios De Silicone Para ...,Utilidade Doméstica,56.00,56.00,4.7,0.00
4,Cuscuzeiro Individual E Cozi Vapore Polido,Utilidade Doméstica,24.57,24.57,4.5,0.00
...,...,...,...,...,...,...
105,Tênis Adidas Ultra Energy Feminino,Esporte,219.99,219.99,4.7,0.00
106,Tênis Masculino Adidas Ultra Energy,Esporte,199.99,199.99,4.5,0.00
107,Kit Bico de Bomba 5 Peças Adaptador Agulhas Bo...,Esporte,4.90,7.90,4.5,0.38
108,Tênis Mizuno Goya Masculino,Esporte,179.99,329.99,4.9,0.45
